
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0                                                        |
| %security_configuration     |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |

In [2]:
%additional_python_modules paramiko

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.31 and you have 0.30 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Additional python modules to be included:
paramiko


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::648742433166:role/awsglue
Attempting to use existing AssumeRole session credentials.
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 05a70d7e-c826-4834-96ac-68f724cfc2ef
Applying the following default arguments:
--glue_kernel_version 0.30
--enable-glue-datacatalog true
--additional-python-modules paramiko
Waiting for session 05a70d7e-c826-4834-96ac-68f724cfc2ef to get into ready status...
Session 05a70d7e-c826-4834-96ac-68f724cfc2ef has been created




In [2]:
import pandas as pd
import datetime
from collections import defaultdict
import paramiko

request = (
    f"-q select * from flights limit 5"
)

username = "ewibrahim"
password = "azerty123"

p = paramiko.SSHClient()
p.set_missing_host_key_policy(paramiko.AutoAddPolicy())
p.connect("data.opensky-network.org", port=2230, username=username, password=password)
stdin, stdout, stderr = p.exec_command(request)
opt = stdout.readlines()

columns = [
            "icao24","firstseen","lastseen","duration","callsign","departure","destination"
]

def get_line_lst(line_txt):
    lst = line_txt.split("|")
    filter_lst = [elt.strip() for elt in lst]
    while filter_lst[0] == "":
        filter_lst = filter_lst[1:]
    while filter_lst[-1] == "":
        filter_lst = filter_lst[:-1]
    return filter_lst

lst_of_lst = []
for i in range(3, len(opt) - 1):
    if opt[i][:2] != "+-":
        l = get_line_lst(opt[i])
        if len(l) != len(columns):
            print("Error in parsing line: ")
            print(l)
            print(opt[i])
        lst_of_lst.append(l)

df2 = pd.DataFrame(lst_of_lst, columns=columns)
df2.dropna(inplace=True)
df2 = df2[df2["callsign"] != "callsign"]
df2 = df2[df2["callsign"] != ""]

df2.to_json('s3://aws-glue-assets-648742433166-us-east-1/notebooks/', orient='records')

FileNotFoundError: [Errno 2] No such file or directory: 's3://aws-glue-assets-648742433166-us-east-1/notebooks/'
